In [ ]:
# fork this repo https://github.com/eric-mitchell/direct-preference-optimization/tree/main

!git clone https://github.com/eric-mitchell/direct-preference-optimization.git


In [ ]:
%%capture
!pip install peft
!pip install hydra-core
!pip install tensor_parallel
!pip install --upgrade datasets

In [ ]:

!huggingface-cli login

In [ ]:
#Command to Run DPO using model obtained from supervised fine tuning.

#Wait for final output from abouve command. Copy the path that looks similar to this (needs to end with policy.pt):
#cache/root/imdb_gpt2_sft_2024-09-15_02-20-25_838729/LATEST/policy.pt

#place this as the argument for model.archive

In [ ]:
import transformers, peft, torch

In [ ]:
from huggingface_hub import hf_hub_download
tokenizer = transformers.AutoTokenizer.from_pretrained('gpt2-large')
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id
# tokenizer.pad_token = tokenizer.eos_token

huggingface_filepath = hf_hub_download(repo_id="your/huggingface/model", filename="policy.pt")
test = transformers.AutoModelForCausalLM.from_pretrained('gpt2-large')
test.load_state_dict(torch.load(huggingface_filepath, map_location=torch.device('cuda'))['state'])

In [ ]:
huggingface_filepath

In [ ]:
cd direct-preference-optimization


In [ ]:
!python -u train.py model=blank_model model.name_or_path=gpt2-large datasets=[jon] loss=dpo loss.beta=0.1 exp_name=gpt2_dpo_dpsdd model.archive='your/huggingface/model/policy.pt' gradient_accumulation_steps=1 batch_size=4 eval_batch_size=2 trainer=BasicTrainer sample_during_eval=false

In [ ]:
from huggingface_hub import HfApi, HfFolder, CommitOperationAdd
api = HfApi()

username = "your/huggingface/model"
repo_name = "your/huggingface/model"
local_filepath = '.cache/root/gpt2_dpo_dpsdd_2024-11-15_21-00-01_314601/LATEST/policy.pt'
api.create_repo(repo_id=f"{username}/{repo_name}",repo_type="model",exist_ok=True,private=True)
api.upload_file(
    path_or_fileobj=local_filepath,
    path_in_repo="policy.pt",
    repo_id=f"{username}/{repo_name}",
    repo_type="model",
)